# Stochastic block model analysis using `graph-tool`

**Warning:** This notebook can only run on a Linux (or Mac OS) as the `graph-tool` module is not available on Windows

In [1]:
import graph_tool.all as gt
import sys
sys.path.append(r"../")

from analysis.analysis_utils import *
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# Loading network from file
filepath = r"../out/memory_test/E3O2P21R12T17_S75_T0.6_M10000_N1000000_init/E3O2P21R12T17_S75_T0.6_M10000_N1000000_init.h5"
adj_matrices, parameters = extract_all_info_from_hdf5(filepath)
niter = parameters["Number of interaction"]
size = parameters["Community size"]
phenotype_table = get_phenotype_table_from_parameters(parameters)

In [3]:
# Network to graph tool graph
edge_list = generate_edge_list_from_matrices(adj_matrices["peTotal"], adj_matrices["link"])
g = gt.Graph()
etrust = g.new_edge_property("double")
elink = g.new_edge_property("int")
g.edge_properties["trust"] = etrust
g.edge_properties["link"] = elink
g.add_edge_list(edge_list, eprops=[g.ep.trust, g.ep.link])

ph = g.new_vertex_property("string", vals=phenotype_table)
g.vertex_properties["Phenotype"] = ph

In [6]:
state_ndc = gt.minimize_nested_blockmodel_dl(g, state_args=dict(deg_corr=False))
state_dc  = gt.minimize_nested_blockmodel_dl(g, state_args=dict(deg_corr=True)) # By default

print("Non degree corrected entropy: ", state_ndc.entropy())
print("Degree corrected entropy: ", state_dc.entropy())
print("With - without: ", state_dc.entropy() - state_ndc.entropy())

state = state_dc

Non degree corrected entropy:  5624.08104665198
Degree corrected entropy:  5483.885577218024
With - without:  -140.19546943395562


In [5]:
# For imporving the inference if needed

S1 = state.entropy()
S = np.zeros(1001)
S[0] = state.entropy()
for i in tqdm(range(1000)): # this should be sufficiently large
   state.multiflip_mcmc_sweep(beta=np.inf, niter=1000)
   S[i+1] = state.entropy()

plt.plot(S)
plt.show()

  0%|          | 0/1000 [00:00<?, ?it/s]

  1%|▏         | 14/1000 [00:21<25:24,  1.55s/it]


KeyboardInterrupt: 

Tab10 colors:
- tab:blue : #1f77b4
- tab:orange : #ff7f0e
- tab:green : #2ca02c
- tab:red : #d62728
- tab:purple : #9467bd

In [7]:
# Setting color based on phenotype
color_dict = {
    "Envious": 'tab:blue',
    "Optimist": 'tab:orange',
    "Pessimist": 'tab:green',
    "Random": 'tab:red',
    "Trustful": 'tab:purple'
}
vcolors = []
for i in range(len(phenotype_table)):
    vcolors.append(color_dict[phenotype_table[i]])
vcolorp = g.new_vertex_property("string", vals=vcolors)

In [8]:
# Setting edges appearence
esizes = []
for i, j, t, l in edge_list:
    esizes.append(float(l))
esizep = g.new_edge_property('float', vals=esizes)

In [12]:
img = state.draw(
            output="../figure/sbm_rl.png", 
            vprops = {
                "size": 6,
                "halo": True,
                "halo_color": vcolorp,
                "halo_size": 1.8, 
                "color": [0, 0, 0, 0.5]
            },
            eprops= {
                "pen_width": esizep
            },
            output_size=(300, 300)
)